In [1]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Merge
from keras import backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [5]:
EMBEDDING_FILE = 'wiki.nl/wiki.nl.vec'
vocabulary = dict()
inverse_vocabulary = ['<unk>']  
# '<unk>' will never be used, it is only a placeholder for the 
# [0, 0, ....0] embedding

print("loading word2vec")
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE,binary=False)
print("done loading word2vec")

loading word2vec
done loading word2vec


In [6]:
#TRAIN_CSV = "dutch_data/dutch_formatted_small.csv"
PREDICT_CSV = "dutch_data/predict_test.csv"

EMBEDDING_FILE = 'wiki.nl/wiki.nl.vec'


predict_df = pd.read_csv(PREDICT_CSV)
print(predict_df)

                                            question1  \
0                      Het moet altijd goed weer zijn   
1   Ik vind dat het wegdek in de nieuwstraat moet ...   
2   het wegdek in de nieuwstraat moet hersteld worden   
3   ik stel voor dat er een volwaardige mantelzorg...   
4   ik stel voor dat er een volwaardige mantelzorg...   
5   ik stel voor dat iedere werknemer een bonus kr...   
6   ik stel voor dat iedere werknemer een bonus kr...   
7                   ik wil dat iedereen opslag krijgt   
8                                ik wil 300 euro meer   
9                                ik wil 300 euro meer   
10  ik stel voor dat er mogelijkheid is tot nachto...   

                                            question2  
0                      Het moet altijd goed weer zijn  
1   het wegdek in de nieuwstraat moet vernieuwd wo...  
2   Het nachtlawaai in de nieuwstraat moet beperkt...  
3   Ik weet dat in Lo reeds een mantelzorgpremie b...  
4   ik stel voor dat mantelzorgers 

In [7]:
def substitute_thousands(text):
    matches = re.finditer(r'[0-9]+(?P<thousands>\s{0,2}k\b)', text, flags=re.I)
    result = ''
    len_offset = 0
    for match in matches:
        result += '{}000'.format(text[len(result)-len_offset:match.start('thousands')])
        len_offset += 3 - (match.end('thousands') - match.start('thousands'))
    result += text[len(result)-len_offset:]
    return result

In [8]:
nltk.download('stopwords')
print("begin sets")
stops = set(stopwords.words('dutch'))
print("end sets")

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = substitute_thousands(text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)

    text = text.split()
    #print(text)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/JorgeDeCorte/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
begin sets
end sets


In [9]:
questions_cols = ['question1', 'question2']



# Iterate over the questions only of both training and test datasets
for dataset in [predict_df]:
    for index, row in dataset.iterrows():
        print(row)
        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):
                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace questions with lists of word indices
            dataset.set_value(index, question, q2n)
            

embedding_dim = 300
# This will be the embedding matrix
embeddings = 1 * np.random.randn(208423, embedding_dim)  
embeddings[0] = 0  # So that the padding will be ignored


# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)      

question1    Het moet altijd goed weer zijn
question2    Het moet altijd goed weer zijn
Name: 0, dtype: object
question1    Ik vind dat het wegdek in de nieuwstraat moet ...
question2    het wegdek in de nieuwstraat moet vernieuwd wo...
Name: 1, dtype: object
question1    het wegdek in de nieuwstraat moet hersteld worden
question2    Het nachtlawaai in de nieuwstraat moet beperkt...
Name: 2, dtype: object
question1    ik stel voor dat er een volwaardige mantelzorg...
question2    Ik weet dat in Lo reeds een mantelzorgpremie b...
Name: 3, dtype: object
question1    ik stel voor dat er een volwaardige mantelzorg...
question2    ik stel voor dat mantelzorgers een financiële ...
Name: 4, dtype: object
question1    ik stel voor dat iedere werknemer een bonus kr...
question2    ik stel voor dat iedere werknemer een financië...
Name: 5, dtype: object
question1    ik stel voor dat iedere werknemer een bonus kr...
question2    ik stel voor dat iedere werknemer tweehonderd ...
Name: 6, dtype: ob

In [10]:
max_seq_length = 243
X = predict_df[questions_cols]
# Split to dicts
X_predict = {'left': predict_df.question1, 'right': predict_df.question2}

print(X_predict)

# Zero padding
for dataset, side in itertools.product([X_predict], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

print(X_predict)
# Make sure everything is ok
assert X_predict['left'].shape == X_predict['right'].shape

{'left': 0                                    [1, 2, 3, 4, 5, 6]
1               [7, 8, 9, 1, 10, 11, 12, 13, 2, 14, 15]
2                        [1, 10, 11, 12, 13, 2, 14, 15]
3     [7, 20, 21, 9, 22, 23, 24, 25, 26, 27, 28, 29,...
4     [7, 20, 21, 9, 22, 23, 24, 25, 26, 27, 28, 29,...
5        [7, 20, 21, 9, 80, 81, 23, 76, 82, 26, 83, 28]
6        [7, 20, 21, 9, 80, 81, 23, 76, 82, 26, 84, 28]
7                                [7, 86, 9, 87, 88, 82]
8                                   [7, 86, 89, 28, 85]
9                                   [7, 86, 89, 28, 85]
10    [7, 20, 21, 9, 22, 95, 66, 96, 97, 98, 99, 11,...
Name: question1, dtype: object, 'right': 0                                    [1, 2, 3, 4, 5, 6]
1                        [1, 10, 11, 12, 13, 2, 16, 15]
2                    [1, 17, 11, 12, 13, 2, 18, 15, 19]
3     [7, 42, 9, 11, 43, 44, 23, 25, 45, 46, 47, 48,...
4            [7, 20, 21, 9, 34, 23, 75, 76, 77, 78, 79]
5                [7, 20, 21, 9, 80, 81, 23, 75, 76, 82

In [14]:
from keras.layers import Bidirectional
# Model variables
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 256
n_epoch = 2

#del malstm
K.clear_session()

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Merge(mode=lambda x: exponent_neg_manhattan_distance(x[0], x[1]), output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

malstm.load_weights("weights-improvement-34-0.79.hdf5")

prediction = malstm.predict([X_predict['left'], X_predict['right']])
#print(prediction)
#print(predict_df)

del malstm
# Adadelta optimizer, with gradient clipping by norm
#optimizer = Adadelta(clipnorm=gradient_clipping_norm)

#malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

/Users/JorgeDeCorte/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:32: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [15]:
predict_to_print = pd.read_csv(PREDICT_CSV)
for index, row in predict_to_print.iterrows():
    print(row[0])
    print(row[1])
    pred = prediction[index][0]
    print(pred)
    if pred > 0.5:
        print("Dubbele vraag gevonden!")
    else:
        print("Niet dubbel...")
    print("\n")

Het moet altijd goed weer zijn
Het moet altijd goed weer zijn
1.0
Dubbele vraag gevonden!


Ik vind dat het wegdek in de nieuwstraat moet hersteld worden
het wegdek in de nieuwstraat moet vernieuwd worden
0.681675
Dubbele vraag gevonden!


het wegdek in de nieuwstraat moet hersteld worden
Het nachtlawaai in de nieuwstraat moet beperkt worden!
0.562189
Dubbele vraag gevonden!


ik stel voor dat er een volwaardige mantelzorgpremie van 50 euro per maand wordt ingevoerd voor alle mantelzorgers in onze gemeente die zorgen voor een zwaar zorgbehoevende persoon
Ik weet dat in Lo reeds een mantelzorgpremie bestaat, doch  Ziekenzorg CM  vraagt deze te verhogen.Deze premie moet administratief makkelijk te bekomen zijn aan de hand van soepele voorwaarden die zo weinig mogelijk mantelzorgers uitsluiten.Deze maatregel is de goedkoopste en meest efficente om de thuiszorg in de toekomst te organiseren.
0.11879
Niet dubbel...


ik stel voor dat er een volwaardige mantelzorgpremie van 50 euro per maand